# Video compression

According to the dataset paper, 

> However, for each target video in the test set, we randomly selected two clips out of three and applied augmentations that
approximate actual degradations seen in real-life video distributions. Specifically, these augmentations were
> 1. reduce the FPS of the video to 15
> 2. reduce the resolution of the video to 1/4 of its original size
> 3. reduce the overall encoding quality

In this notebook we look at how to compress videos using `ffmpeg`.  Based on the above list, a quick search returns several useful examples:

* General compression for the web 1: https://dev.to/benjaminblack/use-ffmpeg-to-compress-and-convert-videos-458l  
* General compression for the web 2: https://trac.ffmpeg.org/wiki/Encode/YouTube   
* Frame rate change: https://trac.ffmpeg.org/wiki/ChangingFrameRate  
* Scaling: https://trac.ffmpeg.org/wiki/Scaling  

The latest `ffmpeg` can be obtained at https://johnvansickle.com/ffmpeg/.  See the FAQ for how to install.

In [1]:
#export
from IPython.display import HTML
import cv2
from fastai.core import *
from fastai.vision import *
from kgl_deepfake.data import *

In [3]:
SOURCE = Path('../data/dfdc_train_part_35//')

In [4]:
fnames = get_files(SOURCE, extensions=['.mp4'])

In [5]:
fn = fnames[159]
fn

PosixPath('../data/dfdc_train_part_35/srvkzbpbnx.mp4')

In [6]:
HTML(html_vid(fn))

Use `mediainfo` to show video properties

In [7]:
! mediainfo {fn}

General
Complete name                            : ../data/dfdc_train_part_35/srvkzbpbnx.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 4.88 MiB
Duration                                 : 10 s 22 ms
Overall bit rate                         : 4 086 kb/s
Encoded date                             : UTC 1904-01-01 00:00:00
Tagged date                              : UTC 1904-01-01 00:00:00
Writing application                      : Lavf57.71.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L4
Format settings, CABAC                   : Yes
Format settings, ReFrames                : 4 frames
Codec ID                                 : avc1
Codec ID/Info        

### File size

To conveniently check the file size of compressed videos:

In [8]:
#export
def get_file_size(fpath):
    assert fpath.is_file()
    p = subprocess.run(f"du -hs {fpath}".split(), stdout=subprocess.PIPE)
    return p.stdout.decode().split('\t')[0]

In [9]:
#export
Path.file_size = get_file_size

Original file size

In [10]:
fn.file_size()

'4.9M'

### Decorator to run command in terminal

In [11]:
#export
def runnit(f):
    def _func(*args, **kwargs):
        command = f(*args, **kwargs)
        p = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return p
    return _func

### Are these videos HDR?

The following checks whether the video is HDR or not.

In [12]:
#export
@runnit
def vid_colour_primaries(fpath):
    "Use `mediainfo` to get video's colour primaries."
    return f'''
    mediainfo f'{fpath}' --Inform="Video;%colour_primaries%"
    '''

def vid_is_HDR(fpath):
    "Is video HDR or not?"
    p = vid_colour_primaries(fpath)
    res = p.stdout.decode()
    return True if 'BT.2020' in res else False

In [13]:
vid_is_HDR(fn)

False

### Show video properties using `mediainfo`

In [14]:
#export
def get_mediainfo(fpath=None): 
    @runnit
    def _func(fpath=None):
        return f'''mediainfo {fpath}'''
    p = _func(fpath=fpath)
    return p.stdout.decode()

### Encoding

Each of the URLs above does something different: encoding, scaling, fps.  These are combined into a single function.  Note that probably not all combinations of input parameters will run, but the main parameters to adjust are:

* **crf** "The range of the CRF scale is 0–51, where 0 is lossless, 23 is the default, and 51 is worst quality possible. A lower value generally leads to higher quality, and a subjectively sane range is 17–28. Consider 17 or 18 to be visually lossless or nearly so; it should look the same or nearly the same as the input but it isn't technically lossless." > 23 recommended.
* **fps** "Changing frame rates requires the video to be re-encoded. Without setting appropriate output quality or bit rate, the video quality may be degraded. Please look at the respective encoding guides for the codec you've chosen."
* **scale** Value between 0 and 1 to scale the original lengths.

To display the video and its properties after it's been compressed

In [15]:
#export
def show_vid_info(f):
    "Display video and its properties after it's been processed."
    def _f(*args, **kwargs):
        p = f(*args, **kwargs)
        fpath = Path(kwargs['fpath_to'])
        display(HTML(html_vid(fpath)))
        print(get_mediainfo(fpath))
    return _f

Function for donig the encoding using `ffmpeg`, with default arguments defined.

In [40]:
#export
def _ffmpeg_web_defaults():
    '''
    These are some reasonable values for uploading. i.e. YouTube, etc.
    '''
    return dict(video_encoder='libx264', video_bitrate='1.5M', fps=30, scale=.5, crf=23, #17-28
                audio_encoder='aac', audio_bitrate='128k')

def _ffmpeg_defaults():
    return dict(video_encoder=None, video_bitrate=None, fps=None, scale=None, crf=None,
                audio_encoder=None, audio_bitrate=None)

def _ffmpeg_fmts():
    "ffmpeg options syntax"
    return dict(video_encoder='-c:v {video_encoder:s}', 
                video_bitrate='-b:v {video_bitrate:s}', 
                fps='-r {fps:d}', 
                scale='-vf scale=iw*{scale:.2f}:ih*{scale:.2f}', 
                crf='-crf {crf:d}',
                audio_bitrate='-b:a {audio_bitrate}', 
                audio_encoder='-c:a {audio_encoder}')

#@show_vid_info
@runnit
def run_ffmpeg(fpath_from=None, fpath_to=None, **kwargs):
    '''
    Run ffmpeg
    '''
    ps = _ffmpeg_defaults()
    ps.update(kwargs)
    pstr = []
    for n, s in _ffmpeg_fmts().items():
        if ps[n] is None: pstr.append('')
        else: pstr.append(s.format(**ps))
    pstr = ' '.join(pstr)
    return f'ffmpeg -i {fpath_from} {pstr} {fpath_to}'

Apply to the original (uncomment the `@show_vid_info` above `run_ffmpeg` to display legible output in the next cell):

In [19]:
f = Path(f'enc_{fn.name}')
if f.exists(): os.remove(f)
run_ffmpeg(fpath_from=fn, fpath_to=f, crf=28)

ffmpeg -i ../data/dfdc_train_part_35/srvkzbpbnx.mp4     -crf 28   enc_srvkzbpbnx.mp4


General
Complete name                            : enc_srvkzbpbnx.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 2.29 MiB
Duration                                 : 10 s 27 ms
Overall bit rate                         : 1 918 kb/s
Encoded date                             : UTC 1904-01-01 00:00:00
Tagged date                              : UTC 1904-01-01 00:00:00
Writing application                      : Lavf58.38.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L4
Format settings, CABAC                   : Yes
Format settings, ReFrames                : 4 frames
Codec ID                                 : avc1
Codec ID/Info                            : A

Double check the properties using `cv2`

In [20]:
vcap = cv2.VideoCapture(str(f))
vlen = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, frame = vcap.read()
vcap.release()
vlen, frame.shape

(300, (1080, 1920, 3))

## Generate compressed sets

Use this section to generate sets of compressed videos.  To keep things simple to begin with, for each directory of original videos, generate 3 compressed sets: one at `fps=15`, one at quarter the size, `scale=.5`, and one with poorer encoding quality, `crf=28`.

In [41]:
SOURCE = Path('../data')

In [42]:
DFDC = Path('dfdc_train_part_35')

In [43]:
ENC_DICT = dict(fps15={'fps':15}, quartersize={'scale':.5}, crf28={'crf':28})

In [47]:
def origfake_exist(row):
    fake_exists = row.fname.exists() 
    orig_exists = True if pd.isna(row.original) else row.original.exists()
    return (fake_exists and orig_exists)

def load_dfdc_json(path, dfdc, fpath=False, drop_missing=False):
    a = get_files(path/dfdc, extensions=['.json'])[0]
    a = pd.read_json(a).T
    a.reset_index(inplace=True)
    a.rename({'index':'fname'}, axis=1, inplace=True)
    if fpath or drop_missing:
        a.fname = a.fname.apply(lambda o: path/dfdc/o)
        a.original = a.original.apply(lambda o: np.nan if pd.isna(o) else path/dfdc/o)
    if drop_missing: 
        exists = a.apply(origfake_exist, axis=1)
        return a[exists]
    return a
    
def compress_dataset(path, dfdc, enc, copy_json=True):
    annots = load_dfdc_json(path, dfdc, drop_missing=True)
    fn_froms = list(annots.fname)
    dir_to = path/f'{dfdc}_{enc}'
    os.makedirs(dir_to, exist_ok=True)
    fn_tos = [dir_to/o.name for o in fn_froms]
    for i in progress_bar(range(len(fn_froms))):
        fpath_from, fpath_to = fn_froms[i], fn_tos[i]
        if fpath_to.exists(): os.remove(fpath_to)
        run_ffmpeg(fpath_from=fpath_from, fpath_to=fpath_to, **ENC_DICT[enc])
    if copy_json: shutil.copy(path/dfdc/'metadata.json', dir_to/'metadata.json')
    return dir_to

**Remember to comment out the `@show_vid_info` to avoid lots of output printed in the next cell.**

In [48]:
! du -hs ../data/dfdc_train_part_35_quartersize/

1.4G	../data/dfdc_train_part_35_quartersize/


In [49]:
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1       197G  111G   79G  59% /


In [50]:
SOURCE.ls()

[PosixPath('../data/dfdc_train_part_0'),
 PosixPath('../data/train_sample_faces'),
 PosixPath('../data/train_sample_videos'),
 PosixPath('../data/dfdc_train_part_40'),
 PosixPath('../data/dfdc_train_part_35'),
 PosixPath('../data/dfdc_train_part_45'),
 PosixPath('../data/dfdc_train_part_35_quartersize'),
 PosixPath('../data/dfdc_train_part_10'),
 PosixPath('../data/models'),
 PosixPath('../data/dfdc_train_part_5'),
 PosixPath('../data/test_videos'),
 PosixPath('../data/train_sample_faces_160.zip'),
 PosixPath('../data/sample_submission.csv')]

In [52]:
dfdcs = [Path(o) for o in ['train_sample_videos', 'dfdc_train_part_0', 'dfdc_train_part_5', 'dfdc_train_part_10']]
dfdcs

[PosixPath('train_sample_videos'),
 PosixPath('dfdc_train_part_0'),
 PosixPath('dfdc_train_part_5'),
 PosixPath('dfdc_train_part_10')]

In [75]:
for dfdc in dfdcs:
    for enc in ENC_DICT.keys():
        print(f'Compressing {dfdc} with {enc} scheme')
        compress_dataset(SOURCE, dfdc, enc, copy_json=True)

Compressing train_sample_videos with fps15 scheme


Compressing train_sample_videos with quartersize scheme


Compressing train_sample_videos with crf28 scheme


Compressing dfdc_train_part_0 with fps15 scheme


Compressing dfdc_train_part_0 with quartersize scheme


...

Compressing dfdc_train_part_5 with quartersize scheme


Compressing dfdc_train_part_5 with crf28 scheme


Compressing dfdc_train_part_10 with fps15 scheme


Compressing dfdc_train_part_10 with quartersize scheme


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [85]:
fns = get_files(SOURCE/f'{dfdcs[-1]}_fps15', extensions=['.mp4'])
len(fns)

3192

In [87]:
!mediainfo {fns[90]}

General
Complete name                            : ../data/dfdc_train_part_10_fps15/mrizixikhs.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 2.16 MiB
Duration                                 : 10 s 134 ms
Overall bit rate                         : 1 785 kb/s
Encoded date                             : UTC 1904-01-01 00:00:00
Tagged date                              : UTC 1904-01-01 00:00:00
Writing application                      : Lavf58.38.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L4
Format settings, CABAC                   : Yes
Format settings, ReFrames                : 4 frames
Codec ID                                 : avc1
Codec ID/Info 

# -fin

In [ ]:
from nbdev.export import *

In [ ]:
notebook2script()

Converted 00_data.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
Converted 00a_video_compression-Copy1.ipynb.
Converted 00a_video_compression.ipynb.
Converted 01_face_detection-Copy1.ipynb.
Converted 01_face_detection.ipynb.
Converted 01a_faces_probs_examples.ipynb.
Converted 01a_faces_probs_examples_hv.ipynb.
Converted 02_fix_luminosity.ipynb.
Converted 02a_create_faceimage_dataset.ipynb.
Converted 02bis_Create_Dataset.ipynb.
Converted 02c_faces_different_dfdc_zips.ipynb.
Converted 03_models.ipynb.
Converted 04_Baseline_C